In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 2
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 2: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/star/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.04
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0/to_pggan/2000shot/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.4, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/star/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    if (epoch+1) % 100 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.040000
Train | 20/20 | Loss:1.0252 | MainLoss:0.7776 | Alpha:0.1187 | SPLoss:0.3531 | CLSLoss:1.7308 | top1:69.8250 | AUROC:0.0000
Test | 161/20 | Loss:0.3650 | MainLoss:0.3650 | SPLoss:0.3720 | CLSLoss:1.5613 | top1:89.5109 | AUROC:0.9728
Test | 153/20 | Loss:0.3815 | MainLoss:0.3815 | SPLoss:0.3720 | CLSLoss:1.5613 | top1:81.7398 | AUROC:0.9870

Epoch: [2 | 1000] LR: 0.052000
Train | 20/20 | Loss:0.6657 | MainLoss:0.4597 | Alpha:0.1157 | SPLoss:0.3733 | CLSLoss:1.4075 | top1:79.6250 | AUROC:0.0000
Test | 161/20 | Loss:0.2627 | MainLoss:0.2627 | SPLoss:0.3765 | CLSLoss:1.2412 | top1:92.8131 | AUROC:0.9921
Test | 153/20 | Loss:0.3284 | MainLoss:0.3284 | SPLoss:0.3765 | CLSLoss:1.2412 | top1:84.5249 | AUROC:0.9889

Epoch: [3 | 1000] LR: 0.064000
Train | 20/20 | Loss:0.5979 | MainLoss:0.4166 | Alpha:0.1240 | SPLoss:0.3829 | CLSLoss:1.0791 | top1:81.6750 | AUROC:0.0000
Test | 161/20 | Loss:0.2044 | MainLoss:0.2044 | SPLoss:0.3862 | CLSLoss:0.9093 | top1:94.9626 | 

Train | 20/20 | Loss:0.4617 | MainLoss:0.3844 | Alpha:0.1164 | SPLoss:0.5241 | CLSLoss:0.1403 | top1:83.1000 | AUROC:0.0000
Test | 161/20 | Loss:0.2488 | MainLoss:0.2488 | SPLoss:0.5166 | CLSLoss:0.1377 | top1:91.4611 | AUROC:0.9985
Test | 153/20 | Loss:0.2649 | MainLoss:0.2649 | SPLoss:0.5166 | CLSLoss:0.1377 | top1:89.9705 | AUROC:0.9928

Epoch: [24 | 1000] LR: 0.159943
Train | 20/20 | Loss:0.4828 | MainLoss:0.4024 | Alpha:0.1167 | SPLoss:0.5536 | CLSLoss:0.1342 | top1:82.4250 | AUROC:0.0000
Test | 161/20 | Loss:0.1320 | MainLoss:0.1320 | SPLoss:0.5674 | CLSLoss:0.1394 | top1:98.7882 | AUROC:0.9995
Test | 153/20 | Loss:0.4528 | MainLoss:0.4528 | SPLoss:0.5674 | CLSLoss:0.1394 | top1:73.5092 | AUROC:0.9916

Epoch: [25 | 1000] LR: 0.159933
Train | 20/20 | Loss:0.4597 | MainLoss:0.3756 | Alpha:0.1231 | SPLoss:0.5393 | CLSLoss:0.1445 | top1:83.1500 | AUROC:0.0000
Test | 161/20 | Loss:0.1219 | MainLoss:0.1219 | SPLoss:0.5411 | CLSLoss:0.1452 | top1:98.2960 | AUROC:0.9986
Test | 153/20 | L

Train | 20/20 | Loss:0.6198 | MainLoss:0.3745 | Alpha:0.1173 | SPLoss:1.9446 | CLSLoss:0.1460 | top1:83.2250 | AUROC:0.0000
Test | 161/20 | Loss:0.2294 | MainLoss:0.2294 | SPLoss:1.3691 | CLSLoss:0.1533 | top1:92.2741 | AUROC:0.9985
Test | 153/20 | Loss:0.2452 | MainLoss:0.2452 | SPLoss:1.3691 | CLSLoss:0.1533 | top1:91.2844 | AUROC:0.9929

Epoch: [46 | 1000] LR: 0.159544
Train | 20/20 | Loss:0.5349 | MainLoss:0.3870 | Alpha:0.1145 | SPLoss:1.1481 | CLSLoss:0.1488 | top1:82.3750 | AUROC:0.0000
Test | 161/20 | Loss:0.1560 | MainLoss:0.1560 | SPLoss:0.9233 | CLSLoss:0.1454 | top1:97.8505 | AUROC:0.9991
Test | 153/20 | Loss:0.3681 | MainLoss:0.3681 | SPLoss:0.9233 | CLSLoss:0.1454 | top1:80.4751 | AUROC:0.9913

Epoch: [47 | 1000] LR: 0.159517
Train | 20/20 | Loss:0.4885 | MainLoss:0.3742 | Alpha:0.1186 | SPLoss:0.8102 | CLSLoss:0.1536 | top1:83.1250 | AUROC:0.0000
Test | 161/20 | Loss:0.1293 | MainLoss:0.1293 | SPLoss:0.7129 | CLSLoss:0.1471 | top1:98.9097 | AUROC:0.9993
Test | 153/20 | L

Train | 20/20 | Loss:0.4334 | MainLoss:0.3600 | Alpha:0.1160 | SPLoss:0.4825 | CLSLoss:0.1498 | top1:84.0000 | AUROC:0.0000
Test | 161/20 | Loss:0.1492 | MainLoss:0.1492 | SPLoss:0.4821 | CLSLoss:0.1488 | top1:96.3022 | AUROC:0.9982
Test | 153/20 | Loss:0.2955 | MainLoss:0.2955 | SPLoss:0.4821 | CLSLoss:0.1488 | top1:86.0354 | AUROC:0.9953

Epoch: [68 | 1000] LR: 0.015877
Train | 20/20 | Loss:0.4282 | MainLoss:0.3559 | Alpha:0.1150 | SPLoss:0.4794 | CLSLoss:0.1490 | top1:83.8000 | AUROC:0.0000
Test | 161/20 | Loss:0.1725 | MainLoss:0.1725 | SPLoss:0.4704 | CLSLoss:0.1478 | top1:95.1682 | AUROC:0.9978
Test | 153/20 | Loss:0.2688 | MainLoss:0.2688 | SPLoss:0.4704 | CLSLoss:0.1478 | top1:87.8473 | AUROC:0.9956

Epoch: [69 | 1000] LR: 0.015872
Train | 20/20 | Loss:0.4288 | MainLoss:0.3556 | Alpha:0.1182 | SPLoss:0.4712 | CLSLoss:0.1477 | top1:84.7750 | AUROC:0.0000
Test | 161/20 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:0.4719 | CLSLoss:0.1479 | top1:95.3551 | AUROC:0.9979
Test | 153/20 | L

Train | 20/20 | Loss:0.4304 | MainLoss:0.3596 | Alpha:0.1137 | SPLoss:0.4786 | CLSLoss:0.1442 | top1:84.0250 | AUROC:0.0000
Test | 161/20 | Loss:0.1546 | MainLoss:0.1546 | SPLoss:0.4774 | CLSLoss:0.1441 | top1:96.2555 | AUROC:0.9979
Test | 153/20 | Loss:0.3006 | MainLoss:0.3006 | SPLoss:0.4774 | CLSLoss:0.1441 | top1:85.3178 | AUROC:0.9955

Epoch: [90 | 1000] LR: 0.015761
Train | 20/20 | Loss:0.4155 | MainLoss:0.3439 | Alpha:0.1144 | SPLoss:0.4794 | CLSLoss:0.1461 | top1:85.3500 | AUROC:0.0000
Test | 161/20 | Loss:0.1518 | MainLoss:0.1518 | SPLoss:0.4821 | CLSLoss:0.1467 | top1:96.0654 | AUROC:0.9977
Test | 153/20 | Loss:0.2939 | MainLoss:0.2939 | SPLoss:0.4821 | CLSLoss:0.1467 | top1:85.7143 | AUROC:0.9955

Epoch: [91 | 1000] LR: 0.015755
Train | 20/20 | Loss:0.4321 | MainLoss:0.3598 | Alpha:0.1159 | SPLoss:0.4785 | CLSLoss:0.1450 | top1:83.8750 | AUROC:0.0000
Test | 161/20 | Loss:0.1499 | MainLoss:0.1499 | SPLoss:0.4788 | CLSLoss:0.1455 | top1:96.2835 | AUROC:0.9979
Test | 153/20 | L

...

Train | 20/20 | Loss:0.3507 | MainLoss:0.3217 | Alpha:0.4228 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:86.4250 | AUROC:0.0000
Test | 161/20 | Loss:0.1573 | MainLoss:0.1573 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:95.7446 | AUROC:0.9991
Test | 153/20 | Loss:0.2471 | MainLoss:0.2471 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:90.5570 | AUROC:0.9918

Epoch: [574 | 1000] LR: 0.000065
Train | 20/20 | Loss:0.3451 | MainLoss:0.3161 | Alpha:0.4245 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:86.8500 | AUROC:0.0000
Test | 161/20 | Loss:0.1576 | MainLoss:0.1576 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:95.7196 | AUROC:0.9991
Test | 153/20 | Loss:0.2466 | MainLoss:0.2466 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:90.5668 | AUROC:0.9918

Epoch: [575 | 1000] LR: 0.000064
Train | 20/20 | Loss:0.3479 | MainLoss:0.3190 | Alpha:0.4234 | SPLoss:0.0000 | CLSLoss:0.0684 | top1:86.5000 | AUROC:0.0000
Test | 161/20 | Loss:0.1574 | MainLoss:0.1574 | SPLoss:0.0001 | CLSLoss:0.0684 | top1:95.7383 | AUROC:0.9991
Test | 153/20 |

Train | 20/20 | Loss:0.3466 | MainLoss:0.3177 | Alpha:0.4227 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:87.0250 | AUROC:0.0000
Test | 161/20 | Loss:0.1567 | MainLoss:0.1567 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:95.7726 | AUROC:0.9991
Test | 153/20 | Loss:0.2481 | MainLoss:0.2481 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:90.4948 | AUROC:0.9916

Epoch: [596 | 1000] LR: 0.000059
Train | 20/20 | Loss:0.3498 | MainLoss:0.3209 | Alpha:0.4227 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:86.6750 | AUROC:0.0000
Test | 161/20 | Loss:0.1568 | MainLoss:0.1568 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:95.7757 | AUROC:0.9991
Test | 153/20 | Loss:0.2480 | MainLoss:0.2480 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:90.4980 | AUROC:0.9916

Epoch: [597 | 1000] LR: 0.000059
Train | 20/20 | Loss:0.3477 | MainLoss:0.3188 | Alpha:0.4233 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:86.6000 | AUROC:0.0000
Test | 161/20 | Loss:0.1567 | MainLoss:0.1567 | SPLoss:0.0001 | CLSLoss:0.0683 | top1:95.7788 | AUROC:0.9991
Test | 153/20 |

Train | 20/20 | Loss:0.3566 | MainLoss:0.3279 | Alpha:0.4207 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:86.1750 | AUROC:0.0000
Test | 161/20 | Loss:0.1573 | MainLoss:0.1573 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7477 | AUROC:0.9991
Test | 153/20 | Loss:0.2473 | MainLoss:0.2473 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.5439 | AUROC:0.9916

Epoch: [618 | 1000] LR: 0.000054
Train | 20/20 | Loss:0.3580 | MainLoss:0.3292 | Alpha:0.4203 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:85.6000 | AUROC:0.0000
Test | 161/20 | Loss:0.1571 | MainLoss:0.1571 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7570 | AUROC:0.9991
Test | 153/20 | Loss:0.2476 | MainLoss:0.2476 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.5374 | AUROC:0.9917

Epoch: [619 | 1000] LR: 0.000054
Train | 20/20 | Loss:0.3357 | MainLoss:0.3066 | Alpha:0.4262 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:87.7250 | AUROC:0.0000
Test | 161/20 | Loss:0.1570 | MainLoss:0.1570 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7601 | AUROC:0.9992
Test | 153/20 |

Train | 20/20 | Loss:0.3489 | MainLoss:0.3200 | Alpha:0.4236 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:86.7000 | AUROC:0.0000
Test | 161/20 | Loss:0.1562 | MainLoss:0.1562 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.8069 | AUROC:0.9992
Test | 153/20 | Loss:0.2486 | MainLoss:0.2486 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.4620 | AUROC:0.9917

Epoch: [640 | 1000] LR: 0.000049
Train | 20/20 | Loss:0.3437 | MainLoss:0.3147 | Alpha:0.4245 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:86.9250 | AUROC:0.0000
Test | 161/20 | Loss:0.1566 | MainLoss:0.1566 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7757 | AUROC:0.9991
Test | 153/20 | Loss:0.2481 | MainLoss:0.2481 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.4948 | AUROC:0.9916

Epoch: [641 | 1000] LR: 0.000048
Train | 20/20 | Loss:0.3512 | MainLoss:0.3223 | Alpha:0.4231 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:87.0500 | AUROC:0.0000
Test | 161/20 | Loss:0.1569 | MainLoss:0.1569 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7601 | AUROC:0.9992
Test | 153/20 |

Train | 20/20 | Loss:0.3423 | MainLoss:0.3134 | Alpha:0.4241 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:87.2750 | AUROC:0.0000
Test | 161/20 | Loss:0.1571 | MainLoss:0.1571 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7352 | AUROC:0.9991
Test | 153/20 | Loss:0.2469 | MainLoss:0.2469 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.5603 | AUROC:0.9917

Epoch: [662 | 1000] LR: 0.000044
Train | 20/20 | Loss:0.3374 | MainLoss:0.3083 | Alpha:0.4259 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:87.7250 | AUROC:0.0000
Test | 161/20 | Loss:0.1571 | MainLoss:0.1571 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7321 | AUROC:0.9991
Test | 153/20 | Loss:0.2469 | MainLoss:0.2469 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:90.5603 | AUROC:0.9916

Epoch: [663 | 1000] LR: 0.000043
Train | 20/20 | Loss:0.3389 | MainLoss:0.3098 | Alpha:0.4255 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:87.3250 | AUROC:0.0000
Test | 161/20 | Loss:0.1573 | MainLoss:0.1573 | SPLoss:0.0000 | CLSLoss:0.0683 | top1:95.7165 | AUROC:0.9991
Test | 153/20 |